In [1]:
# Imports
import pathlib
import os
from dataclasses import dataclass, field
from typing import Dict, Literal, Optional, Tuple
import json

import imutils
import numpy as np
import cv2
from dataclasses_json import DataClassJsonMixin
import pandas as pd
from tqdm import tqdm

# Constants 
CWD = pathlib.Path(os.path.abspath(""))
GIT_ROOT = CWD.parent
DATA_DIR = GIT_ROOT / "data" / 'PhotosynthesisFall2022'

In [2]:
# Game state reconstruction routines
@dataclass
class Participant(DataClassJsonMixin):
    id: str
    position: Tuple[float, float]
    state: Literal['null', 'H2O', 'CO2', 'Sugar', 'O2', 'Thinking_H2O'] = 'null'

@dataclass
class EnvironmentState(DataClassJsonMixin):
    sun_state: Optional[bool] = None

@dataclass
class GameState(DataClassJsonMixin):
    participants: Dict[str, Participant] = field(default_factory=dict)
    environment: EnvironmentState = field(default_factory=EnvironmentState)

@dataclass
class GamePhaseState(DataClassJsonMixin):
    phase: Literal['before_game', 'molecule_select', 'normal', 'paused']

In [3]:
# Helper functions
def xy_transforms(xy: Tuple[float, float], w: int, h:int, c: Dict[str, float]) -> Tuple[float, float]:
    x, y = xy
    xx = int(((x+1)*h/2)*c['AFFINE'][0] + c['OFFSET'][0])
    yy = int(((y+1)*w/2)*c['AFFINE'][1] + c['OFFSET'][1])
    return (xx, yy)

def render(game_state: GameState, frame: np.ndarray, c: Dict[str, float]):

    if game_state.participants:
        for p in game_state.participants.values():
            if isinstance(p.position[0], str):
                p.position = (eval(p.position[0]), eval(p.position[1]))
            xy = xy_transforms(p.position, frame.shape[0], frame.shape[1], c)
            frame = cv2.circle(frame, xy, 30, (0, 0, 255), 1)
            frame = cv2.putText(
                frame, 
                p.id, 
                (xy[0]-22, xy[1]-11), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.4, 
                (0,0,0), 
                2, 
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.id, 
                (xy[0]-22, xy[1]-11), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.4, 
                (255,255,255), 
                1, 
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.state, 
                (xy[0]-5*(len(p.state)), xy[1]+13), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.6, 
                (0,0,0), 
                2,
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.state, 
                (xy[0]-5*(len(p.state)), xy[1]+13), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.6, 
                (255,255,255), 
                1,
                cv2.LINE_AA
            )

    return frame

In [4]:
# Load the data
vid_file = DATA_DIR / 'videos' / 'day 13' / "day13-screen-recording-corrected.mp4"
assert vid_file.exists()
cap = cv2.VideoCapture(str(vid_file))
fps = cap.get(cv2.CAP_PROP_FPS)

game_state_file = DATA_DIR / 'time_alignment' / 'day13_aligned_game_state.csv'
assert game_state_file.exists()
game_state_logs = pd.read_csv(game_state_file)

In [8]:
annotated_events_file = DATA_DIR / 'videos' / 'day 13' / 'annotated_game_events.csv'
assert annotated_events_file.exists()
annotated_events = pd.read_csv(annotated_events_file)

# Game logs
TIME_OFFSET = 4*60 + 30.9
game_logs = DATA_DIR / 'logs' / 'VU_GEM-STEP_NB_2022_Fall_AH_GroupB_Day13_221128_ComputerLogs' / 'game_logs.csv'
assert game_logs.exists()
game_logs = pd.read_csv(game_logs)
game_logs = game_logs[game_logs['event_type'] == 'touch'].reset_index(drop=True)
game_logs['datetime'] = pd.to_datetime(game_logs['datetime'], format='%Y-%m-%d %H:%M:%S.%f')
game_logs['timestamp'] = (game_logs['datetime'] - game_logs['datetime'].iloc[0]).dt.total_seconds()
game_logs['timestamp'] = game_logs['timestamp'] - TIME_OFFSET
game_logs = game_logs[game_logs['timestamp'] >= 0].reset_index(drop=True)

# Combine game and annotated events
game_logs = pd.concat([game_logs, annotated_events])
game_logs = game_logs.sort_values('timestamp')

# Create the new game state logs
new_game_state_logs = game_state_logs.copy()

# Transformation constants
START_INDEX = (0)*fps
CORRECTIONS = {'OFFSET': (-252,-280), 'AFFINE': (1.8,2)}
N = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
sprite_map = {
    '24': 'plant_body',
    # '25': 'root',
    # '26': 'rabbit',
    '25': 'plant_body',
    '26': 'leaf',
    '46': 'plant_body',
    '47': 'leaf'
}
RECORD = False

# Reset video and logs
cap.set(cv2.CAP_PROP_POS_FRAMES, START_INDEX)
game_state_pointer = 0
game_event_pointer = -1
game_state = None
game_phase = GamePhaseState(phase='before_game')
molecule_select_time = None

# If you want to save the video
if RECORD:
    video_file = DATA_DIR / 'time_alignment' / f"day13-alignment-with-state.mp4"
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc('F','M','P','4')
    out = cv2.VideoWriter(str(video_file),fourcc,fps,(650,566))

for i in tqdm(range(N), total=N):
    
    ret, frame = cap.read()
    if not ret:
        break

    frame_timestamp = (START_INDEX + i) / fps
    frame = cv2.putText(
        frame, 
        # f"{pd.Timestamp(frame_timestamp, unit='s').strftime('%H:%M:%S.%f')}", 
        f"{frame_timestamp}",
        (5,30), 
        cv2.FONT_HERSHEY_SIMPLEX, 
        1, 
        (0,0,255), 
        1, 
        cv2.LINE_AA
    )

    # Obtaining game state
    while game_state_logs['timestamp'].iloc[game_state_pointer+1] < frame_timestamp:

        # Update the new game state logs
        if game_state:
            new_game_state_logs.at[game_state_pointer, 'state'] = game_state.to_json()

        game_state_pointer += 1

    new_game_state = GameState.from_json(game_state_logs.iloc[game_state_pointer].state)
    if game_state:
        # Update xy
        for p in game_state.participants.values():
            if p.id in new_game_state.participants:
                p.position = new_game_state.participants[p.id].position
        
    else:
        game_state = new_game_state

    # Processing events
    while len(game_logs) > game_event_pointer+1 and game_logs['timestamp'].iloc[game_event_pointer+1] < frame_timestamp:

        event = game_logs.iloc[game_event_pointer+1]
        if event.event_type == 'game_reset':
            game_phase.phase = 'before_game'
            for p in game_state.participants.values():
                p.state = 'O2'

        elif event.event_type == 'state_update':
            data = event.event_data.replace("“", '"')
            data = data.replace("”", '"')
            try:
                data = json.loads(data)
            except:
                print(f"Failing to str2JSON: {data}")
                raise RuntimeError()
            p = game_state.participants[data['id']]
            assert data['state'] in ['H2O', 'CO2', 'Sugar', 'O2', 'Thinking_H2O']
            p.state = data['state']

        elif event.event_type == 'phase':
            data = event.event_data.replace("“", '"')
            data = data.replace("”", '"')
            phase = json.loads(data)['phase']
            assert phase in ['before_game', 'molecule_select', 'normal', 'paused']
            game_phase.phase = phase

            if phase == 'molecule_select':
                molecule_select_time = event.timestamp

        # ERROR: Analyzing the touch events is unreliable :(
        # elif event.event_type == 'touch':
        #     data = json.loads(event.event_data)
        #     if 'pz' in data['src']:
        #         if game_phase.phase == 'normal': # Only allow photosynthesis transitions
                    
        #             # 'H2O', 'CO2', 'Sugar', 'O2', 'Thinking_H2O'
        #             p = game_state.participants[data['src']]

        #             # These two transitions will need manual annotation
        #             # if p.state == 'H2O' and sprite_map[data['dst']] == 'leaf':
        #             #     p.state = 
        #             # elif p.state == 'CO2':
        #             #     ...
                    
        #             if p.state == 'Sugar':
        #                 if sprite_map[data['dst']] == 'root':
        #                     p.state = 'H2O'
        #                 elif sprite_map[data['dst']] == 'leaf' or sprite_map[data['dst']] == 'plant_body':
        #                     p.state = 'Thinking_H2O'
        #             elif p.state == 'O2':
        #                 print(data)
        #                 # if sprite_map[data['dst']] == 'rabbit':
        #                 #     p.state = 'CO2'
        #             elif p.state == 'Thinking_H2O' and sprite_map[data['dst']] == 'root':
                        # p.state = 'H2O'

        game_event_pointer += 1

    # Keep record
    new_game_state_logs.at[game_state_pointer, 'state'] = game_state.to_json()

    # For macro alignment
    # frame = cv2.putText(
    #     frame, 
    #     f"{game_state_pointer}", 
    #     (5,60), 
    #     cv2.FONT_HERSHEY_SIMPLEX,
    #     1, 
    #     (0,0,255), 
    #     1, 
    #     cv2.LINE_AA
    # )
    frame = cv2.putText(
        frame, 
        f"{game_phase.phase}", 
        (5,60), 
        cv2.FONT_HERSHEY_SIMPLEX,
        1, 
        (0,0,255), 
        1, 
        cv2.LINE_AA
    )

    frame = render(game_state, frame, CORRECTIONS)

    cv2.imshow('frame', imutils.resize(frame, width=1000))
    if RECORD:
        out.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
if RECORD:
    out.release()

# Save into a new file
new_game_state_file = DATA_DIR / 'time_alignment' / 'day13_aligned_game_state-complete.csv'
new_game_state_logs.to_csv(new_game_state_file, index=False)